# Peppol Readiness Checker

Upload your file with Name, VAT, and Country to check Peppol registration and VAT validity.

In [ ]:

import pandas as pd
import os

# Try using uploaded file, otherwise load default
try:
    df
except NameError:
    default_file = "sample_peppol_input.csv"
    if os.path.exists(default_file):
        df = pd.read_csv(default_file)
        print("✅ Default file loaded: sample_peppol_input.csv")
        display(df.head())
    else:
        print("⚠️ Please upload your file first in Step 1 before running this cell.")
        raise SystemExit

# Step 1: Upload your .csv or .xlsx file
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import io

upload_widget = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=False
)

display(upload_widget)

def handle_upload(change):
    global df
    for name, file_info in upload_widget.value.items():
        content = io.BytesIO(file_info['content'])
        if name.endswith('.csv'):
            df = pd.read_csv(content)
        else:
            df = pd.read_excel(content)
        print("✅ File uploaded successfully!")
        display(df.head())

upload_widget.observe(handle_upload, names='value')

In [ ]:
# Step 2: Run Peppol & VAT Checks (AFTER file is uploaded)
try:
    df
except NameError:
    print("⚠️ Please upload your file first in Step 1 before running this cell.")
else:
    import requests
    from zeep import Client
    import dns.resolver

    results = []

    for index, row in df.iterrows():
        name = str(row['Name'])
        vat = str(row['VAT']) if pd.notnull(row['VAT']) else None
        country = str(row['Country'])

        peppol_status = "No"
        peppol_id = None
        vat_status = "Not checked"

        if vat and country:
            peppol_id = f"0088:{vat}".lower()
            try:
                domain = f"{peppol_id.replace(':','.')}.sml.peppolcentral.org"
                dns.resolver.resolve(domain, 'A')
                peppol_status = "Yes"
            except:
                peppol_status = "No"
            try:
                wsdl = "http://ec.europa.eu/taxation_customs/vies/checkVatService.wsdl"
                client = Client(wsdl=wsdl)
                result = client.service.checkVat(countryCode=country[:2].upper(), vatNumber=vat)
                vat_status = "Valid" if result['valid'] else "Invalid"
            except:
                vat_status = "Error"
        elif name:
            if "test" in name.lower():
                peppol_status = "Likely"
                peppol_id = "Estimated by Name"
            else:
                peppol_status = "Unknown"
                peppol_id = "N/A"

        results.append({
            "Name": name,
            "VAT": vat,
            "Country": country,
            "Peppol Registered": peppol_status,
            "Peppol ID": peppol_id,
            "VAT Valid": vat_status
        })

    results_df = pd.DataFrame(results)
    results_df